# Agent service 

In [1]:
import dataclasses
from typing import cast, List

import enact
from enact.agents import agent_server
from enact.agents import agents
from enact.agents import environments
from enact.agents import node_api
from enact.agents import tasks
from enact.coroutines import threaded_async

## Tic-Tac-Toe boards

In [2]:
from typing import Optional


@enact.register
@dataclasses.dataclass
class Move(enact.Resource):
  player: str
  x: int
  y: int


@enact.register
@dataclasses.dataclass
class TicTacToeBoard(enact.Resource):
  next_player: str = 'X'
  board: List[List[str]] = dataclasses.field(
      default_factory=lambda: [[' ']*3 for _ in range(3)])

  def winner(self) -> Optional[str]:
    """Returns the winner of the game, or None if the game is not over."""
    for row in range(3):
      if (self.board[row][0] == self.board[row][1] == self.board[row][2] and
          self.board[row][0] != ' '):
        return self.board[row][0]
    for col in range(3):
      if (self.board[0][col] == self.board[1][col] == self.board[2][col] and
          self.board[0][col] != ' '):
        return self.board[0][col]
    for diag in [(0, 0), (0, 2)]:
      if (self.board[diag[0]][diag[1]] == self.board[1][1] ==
          self.board[2-diag[0]][2-diag[1]] and self.board[1][1] != ' '):
        return self.board[1][1]
    return None

  def ended(self) -> bool:
    return self.winner() is not None or self.moves() == []

  def moves(self) -> List[Move]:
    return [Move(player=self.next_player, x=x, y=y)
            for x in range(3) for y in range(3)
            if self.board[y][x] == ' ']

  def draw(self) -> str:
    """Draw the board."""
    return '\n'.join(['|' + '|'.join(row) + '|'
                      for row in self.board])

  def play_move(self, move: Move):
    if self.board[move.y][move.x] != ' ':
      raise ValueError(f'Invalid move: {move}\n{self.draw()}')
    if move.player != self.next_player:
      raise ValueError(f'Invalid player: {move}\n{self.draw()}')
    self.board[move.y][move.x] = move.player
    self.next_player = 'X' if move.player == 'O' else 'O'


## Environment schemas

### TicTacToe player schema

In [3]:
player_schema = node_api.NodeSchema()
player_schema.post_signature = node_api.Signature.from_types(
    Move, None)
player_schema.get_signature = node_api.Signature.from_types(
    None, TicTacToeBoard)
print('Board schema:')
print(player_schema.pformat())

Board schema:
root
  POST(__main__.Move) -> none
  GET(none) -> __main__.TicTacToeBoard


### TicTacToe board schema

In [4]:
board_schema = node_api.NodeSchema()
board_schema.add_attribute('X', player_schema)
board_schema.add_attribute('O', player_schema)
print('Board schema:')
print(board_schema.pformat())

Board schema:
root
  .X
    POST(__main__.Move) -> none
    GET(none) -> __main__.TicTacToeBoard
  .O
    POST(__main__.Move) -> none
    GET(none) -> __main__.TicTacToeBoard


### Global service schema 

In [5]:
service_schema = node_api.NodeSchema()
service_schema.add('gameboards').element_type = board_schema
print('Service schema:')
print(service_schema.pformat())

Service schema:
root
  .gameboards
    /*
      .X
        POST(__main__.Move) -> none
        GET(none) -> __main__.TicTacToeBoard
      .O
        POST(__main__.Move) -> none
        GET(none) -> __main__.TicTacToeBoard


### Defining the TicTacToe player agent

In [6]:
import random

# Agent API:
async def get_board() -> TicTacToeBoard:
  environment = environments.Environment.current()
  return cast(TicTacToeBoard, await environment.get('', None))

async def play_move(move: Move) -> TicTacToeBoard:
  environment = environments.Environment.current()
  return cast(
    TicTacToeBoard,
    await environment.post(f'', move))


# Agent behavior:
async def play_tic_tac_toe(unused_node_id, unused_data) -> str:
  board = await get_board()
  me = board.next_player
  while not board.ended():
    assert board.next_player == me
    moves = board.moves()
    await play_move(random.choice(moves))
    board = await get_board()
  if board.winner() == me:
    return 'Haha, I win!'
  elif board.winner() == ' ':
    return 'A draw...'
  else:
    return 'Boohoo, I lose!'

In [7]:
tic_tac_toe_agent = agents.Agent(
  name='DeepToe',
  environment_schema=player_schema,
  agent_handlers=node_api.AsyncNode(
    post_handler=play_tic_tac_toe,
    post_signature=node_api.Signature.from_types(None, str)))

### Defining a TicTacToe Server

In [8]:
import abc

class EnvironmentServer(abc.ABC):
  """Base class for environment servers."""

  @property
  @abc.abstractmethod
  def schema(self) -> node_api.NodeSchema:
    pass

  @abc.abstractmethod
  def update(self, outstanding_tasks: List[tasks.Task]):
    pass



class TicTacToeEnvironmentServer(EnvironmentServer):
  """A server for a single TicTacToe environment."""

  def __init__(self):
    self._board = TicTacToeBoard()

  @property
  def schema(self) -> node_api.NodeSchema:
    return board_schema

  def update(self, outstanding_tasks: List[tasks.Task]):
    for task in outstanding_tasks:
      node_id = task.node_id
      assert node_id.matches('X') or node_id.matches('O')
      if str(task.node_id) != self._board.next_player:
        # Ignore tasks for non-active players
        continue
      if isinstance(task, tasks.GetTask):
        task.future.set_result(enact.deepcopy(self._board))
      elif isinstance(task, tasks.PostTask):
        move = task.data
        try:
          self._board.play_move(move)
          task.future.set_result(None)
        except Exception as e:
          task.future.set_exception(e)
      else:
        assert False


In [9]:
from typing import Dict

class AgentService(agent_server.AgentServer):

  def __init__(self):
    super().__init__(service_schema)
    self._servers: Dict[node_api.NodeId, EnvironmentServer] = {}

  def add_environment_server(
      self, location: node_api.NodeId, server: EnvironmentServer):
    """Add a new environment to serve a given location."""
    if not node_api.is_subschema(self.schema.find(location), server.schema):
      raise ValueError(
        f'Server schema does not match service schema at {location}:'
        f'\n{server.schema.pformat()}\nvs\n{self.schema.pformat()}')
    self._servers[location] = server

  def update(self):
    super().update()
    per_location = {
      location: [] for location in self._servers.keys()}
    for task in self.outstanding_tasks:
      for location in per_location.keys():
        if sublocation := task.node_id.strip_prefix(location):
          subtask = dataclasses.replace(task, node_id=sublocation)
          per_location[location].append(subtask)
          break
      else:
        raise ValueError(f'No server for task at: {task.node_id}')
    for location, server in self._servers.items():
      server.update(per_location[location])



In [12]:
def run_game(service: AgentService,
             gameboard_id: node_api.NodeId,
             player1: agents.Agent, player2: agents.Agent):
  """Complete a game between two agents."""
  print(f'Starting game at {gameboard_id}')
  player1_location = f'{gameboard_id}.X'
  player2_location = f'{gameboard_id}.O'
  # Start agents at different locations.
  print(f'Deploying agent: {player1.name} at {player1_location}')
  task1 = service.post(player1, player1_location, '', None)
  print(f'Deploying agent: {player2.name} at {player2_location}')
  task2 = service.post(player2, player2_location, '', None)
  # Run the game loop.
  print('Let the games begin...')
  while not task1.done() and not task2.done():
    service.update()
  print(f'Player X: {task1.wait()}')
  print(f'Player O: {task2.wait()}')

In [14]:
with AgentService() as service:
  env_server = TicTacToeEnvironmentServer()
  gameboard_id = node_api.NodeId('gameboards/1')
  service.add_environment_server(gameboard_id, env_server)

  run_game(service, gameboard_id, tic_tac_toe_agent, tic_tac_toe_agent)

Starting game at gameboards/1
Deploying agent: DeepToe at gameboards/1.X
Deploying agent: DeepToe at gameboards/1.O
Let the games begin...


KeyboardInterrupt: 